In [25]:
from dataclasses import dataclass
from pathlib import Path
import os
import joblib
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
from mlProject import logger

In [19]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    X_data_train_dir: Path
    Y_data_train_dir: Path
    X_data_test_dir: Path
    Y_data_test_dir:Path
    report_dir: Path
    model_dir: Path
    

In [20]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml , create_directories
##from mlProject.entity.config_entity import ModelEvaluationConfig 
class ConfigurationManager:
    def __init__(self, config_file_path= CONFIG_FILE_PATH ,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    

    def get_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
        root_dir=config.root_dir,
        X_data_train_dir=config.X_data_train_dir,  
        Y_data_train_dir=config.Y_data_train_dir,
        X_data_test_dir=config.X_data_test_dir,
        Y_data_test_dir=config.Y_data_test_dir, 
        report_dir=config.report_dir,
        model_dir=config.model_dir
        )
        return model_evaluation_config

In [34]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_model(self):
       model=joblib.load(self.config.model_dir)
       X_test = pd.read_csv(self.config.X_data_test_dir)
       Y_test = pd.read_csv(self.config.Y_data_test_dir)
       y_pred = model.predict(X_test)
       r2_scoree=r2_score(Y_test, y_pred)
       mse = mean_squared_error(Y_test, y_pred)

       with open(self.config.report_dir, 'w') as report_file:
           report_file.write(f"R2 Score: {r2_scoree}\n")
           report_file.write(f"Mean Squared Error: {mse}\n")
           logger.info(f"Model evaluation report saved at {self.config.report_dir}")      
       


In [32]:
%pwd

'c:\\Users\\S Kumar\\project\\end_to_end_project_ml'

In [31]:
os.chdir('../')

In [35]:
config=ConfigurationManager()
model_evaluation_config=config.get_evaluation_config()  
model_evaluation=ModelEvaluation(config=model_evaluation_config)
model_evaluation.evaluate_model()   

[2025-04-25 15:08:08,165]: INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-04-25 15:08:08,168]: INFO:common:yaml file: params.yaml loaded successfully]
[2025-04-25 15:08:08,171]: INFO:common:yaml file: schema.yaml loaded successfully]
[2025-04-25 15:08:08,173]: INFO:common:created directory at: artifact]
[2025-04-25 15:08:08,174]: INFO:common:created directory at: artifact/model_evaluation]
[2025-04-25 15:08:08,213]: INFO:421575419:Model evaluation report saved at artifact/model_evaluation/report.txt]
